In [2]:
import os
import sys

# Get the current working directory of the notebook
current_dir = os.getcwd()

# Check if the last part of the path is 'notebooks'
if os.path.basename(current_dir) == 'notebooks':
    # If it is, then change the directory to the parent folder (the project root)
    project_root = os.path.dirname(current_dir)
    os.chdir(project_root)

# Verify the new working directory
print(f"Current Working Directory has been set to: {os.getcwd()}")

# Add the project root to the Python path
# This makes sure Python can find your modules
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

Current Working Directory has been set to: /Users/rchhetri/C360 Projects/C360_metric_importance


In [3]:
# Magic commands to automatically reload modules
%load_ext autoreload
%autoreload 2

# Import your custom modules just like any other library!
from config import SnowflakeConfig
from snowflake_connector import SnowflakeConnector

# Import other necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Initialize your configuration and connector
sf_config = SnowflakeConfig()
connector = SnowflakeConnector(config=sf_config)

# Write your SQL query
query = """
  with filtered_data as 
(
select *
from dev.gtm.customer_360_v5_hscores
where 
 LATEST_EFFECTIVE_ACR between 100000 and 1000000
 and SALES_HIER_GEO in ('AMER')
),
min_accnt_date as 
(
select SUBSCRIPTION_ACCOUNT_ID, SFDC_ACCOUNT_NAME, min(subscription_term_start_date) as min_sub_date
from dev.gtm.customer_360_v5_hscores
group by SUBSCRIPTION_ACCOUNT_ID, SFDC_ACCOUNT_NAME
),
segments as (
select DATEDIFF(MONTH, m.min_sub_date, current_date()) as accnt_age_mths, f.*,
 CASE WHEN DATEDIFF(MONTH, m.min_sub_date, current_date()) <=6 AND latest_buying_program = 'Volume Plan' THEN 'Early Volume Plan'
     WHEN DATEDIFF(MONTH, m.min_sub_date, current_date()) > 6 AND latest_buying_program = 'Volume Plan' THEN 'Late Volume Plan'
     WHEN DATEDIFF(MONTH, m.min_sub_date, current_date()) <= 6 AND latest_buying_program = 'Savings Plan' THEN 'Early Savings Plan'
     WHEN DATEDIFF(MONTH, m.min_sub_date, current_date()) > 6 AND latest_buying_program = 'Savings Plan' THEN 'Late Savings Plan'
     WHEN latest_buying_program = 'PAYG' THEN 'PAYG' ELSE 'Others' END AS segment_name

from filtered_data f
left join min_accnt_date m
on f.SUBSCRIPTION_ACCOUNT_ID = m.SUBSCRIPTION_ACCOUNT_ID
and f.SFDC_ACCOUNT_NAME = m.SFDC_ACCOUNT_NAME
)

select *
from segments
where segment_name in ('Early Volume Plan','Early Savings Plan','Late Volume Plan','Late Savings Plan','PAYG') ;
"""

# Connect, execute the query, and close the connection
try:
    connector.connect()
    df_raw = connector.execute_query(query)
finally:
    connector.close()


 pip install snowflake-connector-python[secure-local-storage]


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://newrelic.okta.com/app/snowflake/exkugjs4xeGHw0Vo10x7/sso/saml?SAMLRequest=lZJRT9swFIX%2FSuQ9J3bSloHVFrVUQCe2dm0AbW9uchu8OnbwdUj493NTimASSHuz7HOuv3vPHZ63pQqewKI0ekTiiJEAdGZyqYsRuU0vw1MSoBM6F8poGJFnQHI%2BHqIoVcUntXvQK3isAV3gC2nk3cOI1FZzI1Ai16IE5C7j68n3G55EjFfWOJMZRd5YPncIRLDOEx4tOUqP9%2BBcxSltmiZqepGxBU0YY5SdUa%2FaS74c9a3v6QN9TFl%2Fr%2FcKL1%2B%2BsE2lPozgM6zNQYT8Ok2X4XKxTkkwOaJeGI11CXYN9klmcLu6OQCgJ9DQWFAy8whhLpwwGKE2zVaJHWSmrGrny0b%2BRLeQU2UK6Tufz0ak2sn8sdgwdlr3gLVq9a1cs0FvM783KdtdMbH81S7axbS5%2FzmtkyIjwd0x2mQf7RyxhrneB%2Br8FUsGITsJ2VmaJLzHeDKI4pP%2BbxLMfKBSC9c5%2F6WOzM6Jjk5UFX0Fp9Du6uIP9lu4um7YnYlZ%2B5UiGroPmBx2hncEdvzfkxjSt%2FaX%2FfvhI5nPlsYXeA4ujS2F%2BzixOIq7G5mH207KoRRSTfLcAqJPTinTXFgQzq%2B5szUQOj78%2Bn7Rx38B&RelayState=ver%3A1-hin

In [5]:
df_raw.head()  # Display the first few rows of the DataFrame

,ACCNT_AGE_MTHS,SUBSCRIPTION_ACCOUNT_ID,EFFECTIVE_SUBSCRIPTION_ACCOUNT_ID,SFDC_ACCOUNT_ID,SFDC_ACCOUNT_NAME,REPORT_AS_OF_DT,REPORT_MONTH,BUYING_PROGRAM,LATEST_BUYING_PROGRAM,EFFECTIVE_ACR,...,SCORE_TYPE,METRIC_SCORE,METRIC_NAME_SCORE,WEIGHTS,ADOPTION_SCORE_CAL,ENGAGEMENT_SCORE_CAL,FINANCIAL_HEALTH_SCORE_CAL,RISK_SCORE_CAL,HEALTH_SCORE,SEGMENT_NAME
0,30,3251820,3251820,0011U00001S8ge2QAB,"Recurly, Inc.",2023-05-31,2023-05-01,Volume Plan,Savings Plan,116000.040000000,...,Adoption_score,NaN,PRODUCT_STICKINESS_RATIO_HSCORE,0,87,0,59,100,61.500000,Late Savings Plan
1,30,3251820,3251820,0011U00001S8ge2QAB,"Recurly, Inc.",2023-05-31,2023-05-01,Volume Plan,Savings Plan,116000.040000000,...,Adoption_score,100.0,USER_STICKINESS_RATIO_HSCORE,0,87,0,59,100,61.500000,Late Savings Plan
2,30,3251820,3251820,0011U00001S8ge2QAB,"Recurly, Inc.",2023-11-30,2023-11-01,Volume Plan,Savings Plan,116000.040000000,...,Financial_health_score,25.0,CONTRACT_LENGTH_SCORE_HSCORE,0,37,0,53,100,47.500000,Late Savings Plan
3,30,3251820,3251820,0011U00001S8ge2QAB,"Recurly, Inc.",2023-01-31,2023-01-01,Volume Plan,Savings Plan,116000.040000000,...,Adoption_score,NaN,PRODUCT_UTILIZATION_RATE_HSCORE,0,31,0,47,100,44.500000,Late Savings Plan
4,30,3251820,3251820,0011U00001S8ge2QAB,"Recurly, Inc.",2023-03-31,2023-03-01,Volume Plan,Savings Plan,116000.040000000,...,Financial_health_score,100.0,MAXIMUM_DAYS_PAST_DUE_HSCORE,0,31,0,60,100,47.750000,Late Savings Plan


In [6]:
df_raw.shape

(272574, 99)

In [7]:
df_raw.columns

Index(['ACCNT_AGE_MTHS', 'SUBSCRIPTION_ACCOUNT_ID',
       'EFFECTIVE_SUBSCRIPTION_ACCOUNT_ID', 'SFDC_ACCOUNT_ID',
       'SFDC_ACCOUNT_NAME', 'REPORT_AS_OF_DT', 'REPORT_MONTH',
       'BUYING_PROGRAM', 'LATEST_BUYING_PROGRAM', 'EFFECTIVE_ACR',
       'IS_CONTRACT_FLAG', 'LATEST_EFFECTIVE_ACR', 'BCM', 'LATEST_BCM',
       'INGEST_BCM', 'INGEST_UNIT_PRICE', 'CCU_BCM', 'CCU_UNIT_PRICE',
       'USERS_BCM', 'USERS_UNIT_PRICE', 'SUBSCRIPTION_TERM_START_DATE',
       'SUBSCRIPTION_TERM_END_DATE', 'RENEWAL_DATE', 'MULTIYEAR_FLAG',
       'TOTAL_ACR', 'INDUSTRY', 'PHYSICAL_COUNTRY', 'SALES_HIER_GEO',
       'SALES_HIER_REGION', 'SALES_HIER_SUB_REGION', 'EMPLOYEES',
       'CHURN_INDICATOR', 'CONTRACT_START_DATE', 'MONTHS_SINCE_CONTRACT_START',
       'DAILY_ENGAGED_USERS', 'DAU_R7D', 'COMMITTED_USERS',
       'ROLLING_60_DAY_MEDIAN', 'DENOM_USERS', 'ADV_USED_FEATURES',
       'TOTAL_ADV_FEATURES', 'DAU_ACTUALS', 'USED_FEATURES',
       'TOTAL_FEATURES_3_PERCENT', 'DAU_R30D', 'TOTAL_FEATURES_R

In [9]:
adoption_score_columns = ['USER_ACTIVATION_PERCENTAGE_HSCORE', 'PRODUCT_STICKINESS_RATIO_HSCORE',
       'USER_STICKINESS_RATIO_HSCORE', 'PRODUCT_UTILIZATION_RATE_HSCORE']
engagement_score_columns = ['ENGAGEMENT_SCORE_HSCORE', 'OPEN_TICKETS_HSCORE','AVG_CSAT_SCORE_HSCORE']
risk_score_columns = ['PCT_P1_CASES_HSCORE', 'PCT_P1P2_CASES_HSCORE', 'RESOLUTION_RATE_HSCORE','CHURN_RISK_SCORE_HSCORE', 'SENTIMENT_SCORE_HSCORE','RENEWAL_URGENCY_MONTHS_HSCORE','MTHLY_FTTR_50_HSCORE']
financial_score_columns = ['REVENUE_GROWTH_HSCORE','OVERAGE_SCORE_HSCORE','CONTRACT_LENGTH_SCORE_HSCORE']

all_hscores = adoption_score_columns + engagement_score_columns + risk_score_columns + financial_score_columns
raw_columns = [col.split('_HSCORE')[0] for col in all_hscores if col.endswith('_HSCORE')]

In [13]:
df_raw[ [raw_columns]] 

KeyError: "None of [Index([('USER_ACTIVATION_PERCENTAGE', 'PRODUCT_STICKINESS_RATIO', 'USER_STICKINESS_RATIO', 'PRODUCT_UTILIZATION_RATE', 'ENGAGEMENT_SCORE', 'OPEN_TICKETS', 'AVG_CSAT_SCORE', 'PCT_P1_CASES', 'PCT_P1P2_CASES', 'RESOLUTION_RATE', 'CHURN_RISK_SCORE', 'SENTIMENT_SCORE', 'RENEWAL_URGENCY_MONTHS', 'MTHLY_FTTR_50', 'REVENUE_GROWTH', 'OVERAGE_SCORE', 'CONTRACT_LENGTH_SCORE')], dtype='object')] are in the [columns]"

In [16]:
raw_columns

['USER_ACTIVATION_PERCENTAGE',
 'PRODUCT_STICKINESS_RATIO',
 'USER_STICKINESS_RATIO',
 'PRODUCT_UTILIZATION_RATE',
 'ENGAGEMENT_SCORE',
 'OPEN_TICKETS',
 'AVG_CSAT_SCORE',
 'PCT_P1_CASES',
 'PCT_P1P2_CASES',
 'RESOLUTION_RATE',
 'CHURN_RISK_SCORE',
 'SENTIMENT_SCORE',
 'RENEWAL_URGENCY_MONTHS',
 'MTHLY_FTTR_50',
 'REVENUE_GROWTH',
 'OVERAGE_SCORE',
 'CONTRACT_LENGTH_SCORE']

In [19]:
df_raw.columns.tolist()

['ACCNT_AGE_MTHS',
 'SUBSCRIPTION_ACCOUNT_ID',
 'EFFECTIVE_SUBSCRIPTION_ACCOUNT_ID',
 'SFDC_ACCOUNT_ID',
 'SFDC_ACCOUNT_NAME',
 'REPORT_AS_OF_DT',
 'REPORT_MONTH',
 'BUYING_PROGRAM',
 'LATEST_BUYING_PROGRAM',
 'EFFECTIVE_ACR',
 'IS_CONTRACT_FLAG',
 'LATEST_EFFECTIVE_ACR',
 'BCM',
 'LATEST_BCM',
 'INGEST_BCM',
 'INGEST_UNIT_PRICE',
 'CCU_BCM',
 'CCU_UNIT_PRICE',
 'USERS_BCM',
 'USERS_UNIT_PRICE',
 'SUBSCRIPTION_TERM_START_DATE',
 'SUBSCRIPTION_TERM_END_DATE',
 'RENEWAL_DATE',
 'MULTIYEAR_FLAG',
 'TOTAL_ACR',
 'INDUSTRY',
 'PHYSICAL_COUNTRY',
 'SALES_HIER_GEO',
 'SALES_HIER_REGION',
 'SALES_HIER_SUB_REGION',
 'EMPLOYEES',
 'CHURN_INDICATOR',
 'CONTRACT_START_DATE',
 'MONTHS_SINCE_CONTRACT_START',
 'DAILY_ENGAGED_USERS',
 'DAU_R7D',
 'COMMITTED_USERS',
 'ROLLING_60_DAY_MEDIAN',
 'DENOM_USERS',
 'ADV_USED_FEATURES',
 'TOTAL_ADV_FEATURES',
 'DAU_ACTUALS',
 'USED_FEATURES',
 'TOTAL_FEATURES_3_PERCENT',
 'DAU_R30D',
 'TOTAL_FEATURES_R7D',
 'USED_FEATURES_R7D',
 'L3M_BCM',
 'DAY_WISE_ACR',
 'M

In [22]:
df_raw[['METRIC_NAME',
 'METRIC_VALUE']]

,METRIC_NAME,METRIC_VALUE
0,PRODUCT_STICKINESS_RATIO,None
1,USER_STICKINESS_RATIO,285.50
2,CONTRACT_LENGTH_SCORE,25.00
3,PRODUCT_UTILIZATION_RATE,None
4,MAXIMUM_DAYS_PAST_DUE,0.00
...,...,...
272569,MTHLY_FTTR_50,None
272570,PRODUCT_STICKINESS_RATIO,36.36
272571,REVENUE_GROWTH,7.38
272572,RENEWAL_URGENCY_MONTHS,None


You should use the raw scores for your regression analysis, not the capped, normalized scores.

The primary reason is that the capping and normalization process intentionally throws away information. By capping scores at the 1st and 99th percentiles, you are treating an extreme outlier the same as a mild outlier (e.g., a score at the 99.1 percentile is treated identically to a score at the 99.9 percentile). This loss of variance can weaken the signal in your data, making it harder for the regression model to detect the true linear relationship. Linear regression thrives on the continuous variance of raw data to find the best fit.

The capping logic is excellent for creating a bounded "health score" for dashboards or business reporting, but for statistical modeling, you want to give the model the most detailed, untransformed information available.

## An Efficient Approach for Feature Importance with 18+ Metrics
Trying to add features one-by-one (forward selection) is very time-consuming and can lead to misleading results, especially with many correlated features. A more robust and efficient approach involves checking for multicollinearity and then using a model that can perform feature selection for you.

Here’s a step-by-step guide:

1. Data Preparation
Start with Raw Scores: Use the raw, uncapped scores for all 18 of your input metrics.
Handle Missing Values: Decide on a strategy for any missing data (e.g., imputation with the mean or median, or more advanced methods).
Standardize Your Data: Before feeding the data into the model, scale all your raw scores. Using a StandardScaler (which transforms data to have a mean of 0 and a standard deviation of 1) is crucial for the next step to work effectively.
2. Check for Multicollinearity
With 18 metrics, it's very likely that some are highly correlated with each other (e.g., "daily active users" and "weekly active users"). High correlation, or multicollinearity, can make p-values and feature importance unreliable.

Calculate a Correlation Matrix: Quickly visualize the relationships between all your raw metrics.
Use Variance Inflation Factor (VIF): This is the standard method. VIF measures how much the variance of an estimated regression coefficient is increased because of collinearity.
Calculate the VIF for each feature.
A common rule of thumb is that a VIF score above 5 or 10 indicates high multicollinearity.
If you find highly correlated features, consider removing one of them from each correlated pair. For example, if two metrics have a VIF of 20 and are highly correlated, keep the one that makes more business sense or has a slightly stronger individual correlation with your target variable.
3. Automated Feature Selection with Lasso Regression
Instead of manually adding features one by one, use Lasso (L1) Regression. Lasso is a type of linear regression that includes a penalty term that forces the coefficients of the least important features to become exactly zero, effectively removing them from the model. This makes it an excellent and highly efficient tool for automated feature selection.

Fit a Lasso Model: Train a Lasso regression model using your standardized raw scores as input and your revenue growth as the target.
Tune the Alpha Parameter: The strength of the Lasso penalty is controlled by a parameter called alpha. You can use cross-validation (LassoCV in Scikit-learn) to find the optimal alpha that provides the best model performance.
Identify Important Features: After fitting the model, inspect the coefficients. The features that still have a non-zero coefficient are the ones the Lasso model has selected as important. The features whose coefficients were shrunk to zero are considered less significant.
4. Final R-squared and P-value Analysis
Now that Lasso has helped you identify the most promising subset of features, you can perform a final, traditional linear regression to get the clear R-squared and p-values you want.

Build a Final Multiple Regression Model: Use only the features that were selected by the Lasso model (those with non-zero coefficients).
Analyze the Output:
R-squared / Adjusted R-squared: This tells you how much of the variance in your revenue growth is explained by your selected features.
P-values: Now you can confidently look at the p-values for each feature in this final model. Because you've already handled multicollinearity and selected the most impactful variables, these p-values will be much more reliable. Any feature with a p-value above your significance threshold (e.g., 0.05) could still be considered for removal for the most parsimonious model.